# Hotel Reviews Classification

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("C:\\Users\\LENOVO\\Desktop\\train.csv")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data['Rating'].value_counts()

In [ ]:
sns.countplot(x='Rating',data=data , palette='rainbow')

In [ ]:
data['text length'] = data['Review'].apply(len)
data.head()

# Text Preprocessing

In [ ]:
# Libraries for text preprocessing
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer, word_tokenize
#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
en_stopwords = stopwords.words('english')

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))
##Creating a list of custom stopwords
new_words = ["using", "show", "result", "large", "also","hotel","n","th", "iv", "one", "two", "new", "previously", "shown"]
stop_words = stop_words.union(new_words)

corpus = []
for index, row in data.iterrows():
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', row['Review'])
    
    #Convert to lowercase
    text = text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    ##Convert to list from string
    text = text.split()
    
    ##Stemming
    ps=PorterStemmer()
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
    text = " ".join(text)
    corpus.append(text)
    data.at[index, 'clean_review'] = text
data

In [ ]:
corpus[0]

# Tokenisation

In [ ]:
from nltk.tokenize import word_tokenize
corpusdata=data['clean_review'].apply(word_tokenize).tolist()
len(corpusdata),data.shape[0]

In [ ]:
corpusdata[0]

In [ ]:
corpusdata = np.asarray(corpusdata)
targets = np.asarray(data['Rating'].tolist())

In [ ]:

corpus1 = data[data['Rating'] == 1]['clean_review'].apply(word_tokenize).tolist()
corpus2 = data[data['Rating'] == 2]['clean_review'].apply(word_tokenize).tolist()
corpus3 = data[data['Rating'] == 3]['clean_review'].apply(word_tokenize).tolist()
corpus4 = data[data['Rating'] == 4]['clean_review'].apply(word_tokenize).tolist()
corpus5 = data[data['Rating'] == 5]['clean_review'].apply(word_tokenize).tolist()

In [ ]:
corpus1[1]

# Unigrams, Bigrams & Trigrams

In [ ]:
def count_ngrams(corpusdata, n=2, ntop=None):
    ngrams = {}
    
    for clean_review in corpusdata:
        clean_review = tuple(clean_review)
        
        for i in range(0, len(clean_review) - n + 1): 
                # Get the n-gram from i to i+n
                ngram = clean_review[i:i+n]
                
                # check if the n-gram is in the dictionary
                if ngram in ngrams:
                    # Increment the count for this n-gram
                    ngrams[ngram] += 1
                else:
                    # Initialize this n-gram count to 1
                    ngrams[ngram] = 1
                    
    ngrams = pd.DataFrame.from_dict(ngrams, orient='index', columns=['count']).sort_values('count', ascending=False)
    
    ngrams.reset_index(inplace=True)
    
    ngrams['index'] = ngrams['index'].str.join(', ')
    
    
    if (ntop):
        ngrams = ngrams[:ntop]    
        
    return ngrams

In [ ]:
uni_grams1 = count_ngrams(corpus1, n=1, ntop=10)
uni_grams2 = count_ngrams(corpus2, n=1,ntop=10)
uni_grams3 = count_ngrams(corpus3, n=1,ntop=10)
uni_grams4 = count_ngrams(corpus4, n=1,ntop=10)
uni_grams5 = count_ngrams(corpus5, n=1,ntop=10)

uni_grams1['Rating'] = 1
uni_grams2['Rating'] = 2
uni_grams3['Rating'] = 3
uni_grams4['Rating'] = 4
uni_grams5['Rating'] = 5

unigrams = pd.concat([uni_grams5, uni_grams4, uni_grams3, uni_grams2, uni_grams1])

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
fig = px.bar(unigrams, x="index", y="count", color="Rating", facet_col="Rating", title='Unigrams of Reviews Rating-wise' ,
             color_continuous_scale=["#EF553B", "#FFA15A", "#FECB52", "#B6E880", "#00CC96"])
fig.update_xaxes(matches=None)
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [ ]:

bi_grams1 = count_ngrams(corpus1,n=2,ntop=10)
bi_grams2 = count_ngrams(corpus2, n=2,ntop=10)
bi_grams3 = count_ngrams(corpus3, n=2,ntop=10)
bi_grams4 = count_ngrams(corpus4, n=2,ntop=10)
bi_grams5 = count_ngrams(corpus5, n=2,ntop=10)

bi_grams1['Rating'] = 1
bi_grams2['Rating'] = 2
bi_grams3['Rating'] = 3
bi_grams4['Rating'] = 4
bi_grams5['Rating'] = 5

bigrams = pd.concat([bi_grams5, bi_grams4, bi_grams3, bi_grams2, bi_grams1])

In [ ]:

fig = px.bar(bigrams, x="index", y="count", color="Rating", facet_col="Rating", title='Bigrams of Reviews Rating-wise' ,
             color_continuous_scale=["#EF553B", "#FFA15A", "#FECB52", "#B6E880", "#00CC96"])
fig.update_xaxes(matches=None)
fig.update_layout(coloraxis_showscale=False)
fig.show()

In [ ]:

tri_grams1 = count_ngrams(corpus1, n=3, ntop=10)
tri_grams2 = count_ngrams(corpus2, n=3, ntop=10)
tri_grams3 = count_ngrams(corpus3, n=3, ntop=10)
tri_grams4 = count_ngrams(corpus4, n=3, ntop=10)
tri_grams5 = count_ngrams(corpus5, n=3, ntop=10)

tri_grams1['Rating'] = 1
tri_grams2['Rating'] = 2
tri_grams3['Rating'] = 3
tri_grams4['Rating'] = 4
tri_grams5['Rating'] = 5

tri_grams = pd.concat([tri_grams5, tri_grams4, tri_grams3, tri_grams2, tri_grams1])

In [ ]:
fig = px.bar(tri_grams, x="index", y="count", color="Rating", facet_col="Rating", title='Trigrams of Reviews Rating-wise',
             color_continuous_scale=["#EF553B", "#FFA15A", "#FECB52", "#B6E880", "#00CC96"])
fig.update_xaxes(matches=None)
fig.update_layout(coloraxis_showscale=False)
fig.show()

# Wordclouds

In [ ]:
from wordcloud import WordCloud
stops = stopwords.words('english') + ['hotel', 'room', 'not']

w1 = WordCloud(color_func=lambda *args, **kwargs: "#EF553B", 
               background_color='white', 
               max_words=50, 
               stopwords=stops,
               random_state=42,
               height=1000, 
               width=1000) \
            .generate(" ".join(data[data['Rating'] == 1]['clean_review']))

w2 = WordCloud(color_func=lambda *args, **kwargs: "#FFA15A", 
               background_color='white', 
               max_words=50, 
               stopwords=stops,
               random_state=42,
               height=1000, 
               width=1000) \
            .generate(" ".join(data[data['Rating'] == 2]['clean_review']))

w3 = WordCloud(color_func=lambda *args, **kwargs: "#FECB52", 
               background_color='white', 
               max_words=50, 
               stopwords=stops,
               random_state=42,
               height=1000, 
               width=1000) \
            .generate(" ".join(data[data['Rating'] == 3]['clean_review']))

w4 = WordCloud(color_func=lambda *args, **kwargs: "#B6E880", 
               background_color='white', 
               max_words=50, 
               stopwords=stops,
               random_state=42,
               height=1000, 
               width=1000) \
            .generate(" ".join(data[data['Rating'] == 4]['clean_review']))

w5 = WordCloud(color_func=lambda *args, **kwargs: "#00CC96", 
               background_color='white', 
               max_words=50,
               stopwords=stops,
               random_state=42,
               height=1000, 
               width=1000) \
            .generate(" ".join(data[data['Rating'] == 5]['clean_review']))

In [ ]:
fig = plt.figure(figsize=(26, 12))

fig.add_subplot(1, 3, 1)
plt.axis('off')
plt.title('Rating=5')
plt.imshow(w5)

fig.add_subplot(1, 3, 2)
plt.title('Rating=4')
plt.axis('off')
plt.imshow(w4)

fig.add_subplot(1, 3, 3)
plt.axis('off')
plt.title('Rating=3')
plt.imshow(w3)

fig = plt.figure(figsize=(26, 12))


fig.add_subplot(1, 2, 1)
plt.title('Rating=2')
plt.axis('off')
plt.imshow(w2)

fig.add_subplot(1, 2, 2)
plt.title('Rating=1')
plt.axis('off')
plt.imshow(w1)

# Named Entity Recognition / Entity Extraction

In [ ]:
import spacy
nlp = spacy.load('en')


Let’s find out the most frequent named entities in our corpus. For this, we will build a data frame consisting of all the named entities and their types.

In [ ]:
named_entities = []
for sentence in corpus:
    temp_entity_name = ''
    temp_named_entity = None
    sentence = nlp(sentence)
    for word in sentence:
        term = word.text 
        tag = word.ent_type_
        if tag:
            temp_entity_name = ' '.join([temp_entity_name, term]).strip()
            temp_named_entity = (temp_entity_name, tag)
        else:
            if temp_named_entity:
                named_entities.append(temp_named_entity)
                temp_entity_name = ''
                temp_named_entity = None

entity_frame = pd.DataFrame(named_entities, 
                            columns=['Entity Name', 'Entity Type'])

To find top occuring entities and their types

In [ ]:
# get the top named entities
top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:20]

Here, we can see that it has correctly identified spanish, italian, american as nationalities. We can also see that night is mostly used.

In [ ]:
# get the top named entity types
top_entities = (entity_frame.groupby(by=['Entity Type'])
                           .size()
                           .sort_values(ascending=False)
                           .reset_index().rename(columns={0 : 'Frequency'}))
top_entities.T.iloc[:,:15]

Here we can see organisation, person, and country/cities/states are the most mentioned entities

# Topic Modelling Using LDA

In [ ]:
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(corpusdata, min_count=10)
trigram = Phrases(bigram[corpusdata])

for idx in range(len(corpusdata)):
    for token in bigram[corpusdata[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            corpusdata[idx].append(token)
    for token in trigram[corpusdata[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            corpusdata[idx].append(token)

In [ ]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(corpusdata)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))


In [ ]:
#Creating term document matrix
corpus_1 = [dictionary.doc2bow(doc) for doc in corpusdata]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus_1))

In [ ]:
# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0] # load dictionary
id2word = dictionary.id2token


model = LdaModel(corpus=corpus_1, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every,random_state=12)

In [ ]:
viz = pyLDAvis.gensim.prepare(model, corpus_1, dictionary)
viz

# Exploring Key attributes

Lets take a look at the terms that appears more in each topic

In [ ]:
def explore_topic(lda_model, topic_number, topn, output=True):
    
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:40} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [ ]:
topic_summaries = []
print(u'{:40} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:10]]

The above four topics can classfied as:

Topic 0 = "Worst Hotels", It includes words like (desk,check,star,booked,told,problem),that make us think that this topic can be related with lower rating of hotels, and seems that those problems are related with reservation problems,reception desk problems,basically the worst hotels.

Topic 1 = "BUsiness Hotels", it includes words like(air_conditioning,mini_bar,coffee,shower). This topic seems to refer Business-class Hotels or something like that.

Topic 2 = "Top/Best/Comfortable Hotels", it includes words like(minute_walk,walking_distance,staff_helpful,comfortable).This means it's related with City Hotels, Close to station/airport or simply located in a major location.

Topic 3 = "Resort Hotels",words like(resort,punta_cana,vacation,trip) clearly shows that it's related to Resort hotels.


In [ ]:
# attach topics to data
all_topics = model.get_document_topics(corpus_1, minimum_probability=0.0)
all_topics_csr = gensim.matutils.corpus2csc(all_topics)
all_topics_numpy = all_topics_csr.T.toarray()
data['Topic'] = all_topics_numpy.argmax(axis=1)

# plot topics distribution by rating
plt.figure(figsize=(8,7))
sns.countplot(data=data,x="Rating",hue="Topic",edgecolor="black",linewidth=3)
plt.legend(['worst Hotels',"Business Hotels","Top Hotels","Resort Hotels"])
plt.title('Topics Distribution by rating',size=18)
plt.show()

# plot topics distribution 
plt.figure(figsize=(8,7))
ax=sns.countplot(data=data,x="Topic",edgecolor="black",linewidth=3)
ax.set_xticklabels(['Worst hotels',"Business Hotels","Top Hotels","Resort Hotels"])
plt.title('Topics Distribution',size=18)
plt.show()